# Recommender Systems
Recommender Systems have become ubiquitous in the modern data science landscape, as companies like Google, Netflix, Pandora, Facebook, etc. rely heavily on them to provide targeted content recommendation to their users to create a more enjoyable user experience.  In these exercises, we'll focus on the process of ***collaborative filtering*** for building recommenders on 2 different datasets (beers and movies).  

[Collaborative Filtering](https://en.wikipedia.org/wiki/Collaborative_filtering) relies on a ***ratings matrix*** for all items to generate similarities between items and users based on similar ratings.  It's important to remember that collaborative filtering is one of the 2 main ways to conduct recommendation, the other being [Content-Based Filtering](https://en.wikipedia.org/wiki/Recommender_system#Content-based_filtering) which explicitly maps items and/or users into a shared feature space based on explicit user/item characteristics.  State of the art recommenders will often rely on hybrid approaches of these 2, so it's important to understand the differences, strengths, and weaknesses of each and what separates them.

### Datasets
- [Beer Ratings](https://github.com/pburkard88/DS_BOS_07/blob/master/Data/beer_reviews.tar.gz): A dataset of beer reviews
- [Movielens Data](https://github.com/pburkard88/DS_BOS_07/blob/master/Data/movielens): A dataset of movie ratings from the original [here](http://grouplens.org/datasets/movielens/)

### Learning Goals
- Perform collaborative filtering from ratings matrices using `pandas` and `sklearn` on the beers data
- Understand why this approach represents collaborative filtering
- Perform collaborative filtering using the [python-recsys](https://github.com/ocelma/python-recsys) library that provides some nice built-in recommender functionality
- Understand how SVDs or other matrix decompositions might fit in in the context of a recommender algorithm

## Similarity based Recommendation System: Beers
The first dataset we'll work with is a list of many beer reviews by a variety of reviewers with accompanying beer metadata on every review.  We'll use this data to generate our reviewer/beer ratings matrix from which we can perform collaborative filtering and recommend beers based on user preferences.

### Beers: Get the Data
First perform the usual imports of `numpy` and `pandas` as `np` and `pd`.

Now let's get the data.  If you don't already have it locally you can use curl to pull it down.

In [3]:
! curl -O https://s3.amazonaws.com/demo-datasets/beer_reviews.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.3M  100 27.3M    0     0  2116k      0  0:00:13  0:00:13 --:--:-- 2220k


These steps here are optional, just move the data some place where you know where it is and then point your eventual call to `read_csv()` to that location.

In [4]:
! mv 'beer_reviews.tar.gz' ../Data/

In [5]:
!ls ../Data

538model                  Heart Disease             beer_reviews.tar.gz       used_vehicles_oos.csv
Amazon Reviews            Spam Classification       lahman-csv_2014-02-14.zip
Bike-Sharing-Dataset      Student_Performance       svm_data.csv
Flight_Delays             Titanic                   used_vehicles.csv


Import the data into a `pandas` dataframe called `df` by calling `read_csv()` with the appropriate path and the parameter `compression='gzip'` (you don't need this if you already extracted your file, it's just nice to see that pandas can handle gzipped data).

/Users/pburkard88/anaconda/envs/python27/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


### Explore the Data
Let's look at the data with `head()`

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


Create a separate data frame `df_test` to investigate a little bit further by selecting out only the **beer_name="Pale Ale"** reviews using the `isIn([])` function.  Then sort this resulting table by **review_profilename** and examine the first 100 rows.  You should notice that the same reviewer can review multiple Pale Ales.

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
912451,19402,Inland Empire Brewing Company,1240528130,4.0,3.0,3.0,0110x011,American Pale Ale (APA),3.0,3.5,Pale Ale,5.50,49291
1406262,9824,Silverado Brewing Company,1253298781,3.5,3.5,3.5,1759Girl,American Pale Ale (APA),2.5,4.0,Pale Ale,5.12,25427
563154,423,Boulevard Brewing Co.,1305678006,3.5,2.0,4.0,1Adam12,American Pale Ale (APA),3.0,3.0,Pale Ale,5.40,2094
525342,2101,Blue Star Brewing Company,1237655645,4.5,4.0,4.0,1fastz28,American Pale Ale (APA),4.0,4.0,Pale Ale,NaN,5828
41264,13397,Mountaineer Brewing Co.,1291941443,4.0,3.0,3.5,321jeff,American Pale Ale (APA),4.0,3.0,Pale Ale,5.59,28951
1385721,3725,Réservoir,1120719056,3.5,3.0,3.0,3Vandoo,English Pale Ale,4.0,3.0,Pale Ale,5.00,24527
562967,423,Boulevard Brewing Co.,1203782439,5.0,4.5,4.5,7thstreetbrewery,American Pale Ale (APA),5.0,4.5,Pale Ale,5.40,2094
563116,423,Boulevard Brewing Co.,1058366048,4.0,3.5,4.0,ADR,American Pale Ale (APA),3.5,3.0,Pale Ale,5.40,2094
1429227,25252,Goodieson Brewery,1304334057,3.0,3.0,3.5,ADZA,American Pale Ale (APA),3.0,3.0,Pale Ale,4.50,68580
836014,18414,Three Troupers Brewery,1288180209,3.5,3.0,3.0,ADZA,English Pale Ale,3.0,3.5,Pale Ale,4.50,46137


Let's restrict this to the top 250 beers. Use the `value_counts()` method to get a sorted list by value count on **beer_name** and then taking the first 250.  Overwrite `df` with this new data.

,beer_reviews/,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
798,1075,Caldera Brewing Company,1212201268,4.5,4.5,4,grumpy,American Double / Imperial Stout,4.0,4.5,Imperial Stout,NaN,42964
1559,11715,Destiny Brewing Company,1137124057,4.0,3.5,4,blitheringidiot,American Pale Ale (APA),3.5,3.5,Pale Ale,4.5,26420
1560,11715,Destiny Brewing Company,1129504403,4.0,2.5,4,NeroFiddled,American Pale Ale (APA),4.0,3.5,Pale Ale,4.5,26420
1563,11715,Destiny Brewing Company,1137125989,3.5,3.0,4,blitheringidiot,American IPA,4.0,4.0,IPA,NaN,26132
1564,11715,Destiny Brewing Company,1130936611,3.0,3.0,3,Gavage,American IPA,4.0,3.5,IPA,NaN,26132


How big is this dataset?  Use `df.info()`

<class 'pandas.core.frame.DataFrame'>
Int64Index: 355275 entries, 798 to 1586564
Data columns (total 13 columns):
beer_reviews/         355275 non-null object
brewery_name          355275 non-null object
review_time           355275 non-null float64
review_overall        355275 non-null float64
review_aroma          355275 non-null float64
review_appearance     355275 non-null float64
review_profilename    355175 non-null object
beer_style            355275 non-null object
review_palate         355275 non-null float64
review_taste          355275 non-null float64
beer_name             355275 non-null object
beer_abv              353477 non-null float64
beer_beerid           355275 non-null float64
dtypes: float64(8), object(5)
memory usage: 37.9+ MB


Aggregate the data in a pivot table called `df_wide` using the `pivot_table` method. Display the mean review_overall for each beer_name aggregating the review_overall values by review_profilename. Use the mean (numpy.mean) as aggregator.  In other words, the `values` parameter should contain **review_overall** and the `index` parameter should contain **beer_name** and **review_profilename**.  Make sure to call `unstack()` at the end.

(250, 22140)

Display the head of the pivot table, but only for 5 users (columns are users)

review_overall                               \
review_profilename               0110x011 02maxima 03SVTCobra 05Harley   
beer_name                                                                
#9                                    NaN      NaN        NaN      NaN   
120 Minute IPA                        NaN      NaN        NaN        4   
1554 Enlightened Black Ale            NaN      NaN        NaN      NaN   
60 Minute IPA                         NaN      NaN        NaN      NaN   
90 Minute IPA                           5      NaN        NaN        4   

                                     
review_profilename         0Naught0  
beer_name                            
#9                              NaN  
120 Minute IPA                  NaN  
1554 Enlightened Black Ale      NaN  
60 Minute IPA                   NaN  
90 Minute IPA                   NaN

### Discussion: what do you notice in this table?

Set Nans to zero with the `fillna()` function.

Check that columns are users by examining the first few columns.

MultiIndex(levels=[[u'review_overall'], [u'0110x011', u'02maxima', u'03SVTCobra', u'05Harley', u'0Naught0', u'0beerguy0', u'0runkp0s', u'0tt0', u'1000Bottles', u'1001111.0', u'100floods', u'100proof', u'1050Sudz', u'108Dragons', u'1099.0', u'1100.0', u'1121987.0', u'11millsown113', u'11osixBrew', u'11thFloorBrewing', u'1229design', u'12NattiBottles', u'12ouncecurls', u'12percent', u'12vUnion', u'130guy', u'13smurrf', u'160Shillings', u'1759Girl', u'1759dallas', u'17Guinness59', u'18alpha', u'196osh', u'1993Heel', u'1996StrokerKid', u'1Adam12', u'1MiltonWaddams', u'1PA', u'1after909', u'1brbn1sctch1beer', u'1fastz28', u'1morebeer', u'1noa', u'1quiks10', u'1santore', u'1thinmint', u'1whiskey', u'20ozmonkey', u'21mmer', u'220emaple', u'22ozStone', u'2369.0', u'2378GCGTG', u'23fyerfyter', u'2408bwk', u'245Trioxin', u'24Beer92', u'25santurce', u'28Rock', u'2BDChicago', u'2Cruzy', u'2DaMtns', u'2KHokie', u'2LBrew', u'2Shay', u'2Stout4u', u'2beerdogs', u'2cansam', u'2girls1hop', u'2heartedMIK

Check that rows are beers by examining the first few rows.

0                                    #9
1                        120 Minute IPA
2            1554 Enlightened Black Ale
3                         60 Minute IPA
4                         90 Minute IPA
5    Aecht Schlenkerla Rauchbier Märzen
6                          AleSmith IPA
7               AleSmith Speedway Stout
8                        Allagash White
9                   Alpha King Pale Ale
Name: beer_name, dtype: object

### Calculate distance between beers

This is the key.  We have our ratings matrix now and we're going to use cosine_similarity from scikit-learn to compute the distance between all beers in this space.

Import stuff

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances

Apply `cosine_similarity()` to `df_wide` to calculate pairwise distances and store this in a variable called `dists`.

array([[ 1.        ,  0.27540494,  0.27410345, ...,  0.32928048,
         0.34805798,  0.31249922],
       [ 0.27540494,  1.        ,  0.25151873, ...,  0.2854835 ,
         0.23301356,  0.2802485 ],
       [ 0.27410345,  0.25151873,  1.        , ...,  0.31629515,
         0.22521858,  0.2737628 ],
       ..., 
       [ 0.32928048,  0.2854835 ,  0.31629515, ...,  1.        ,
         0.28025764,  0.34504013],
       [ 0.34805798,  0.23301356,  0.22521858, ...,  0.28025764,
         1.        ,  0.25526913],
       [ 0.31249922,  0.2802485 ,  0.2737628 , ...,  0.34504013,
         0.25526913,  1.        ]])

### Discussion: what type of object is dists?

Convert dists to a Pandas DataFrame, use the index as column index as well (distances are a square matrix).  This means we'll have a beers by beers matrix of the distances between every beer from the ratings space.  Check out the first 10 or so rows and columns and make sure things look right (should see 1s on the diagonal).

beer_name,#9,120 Minute IPA,1554 Enlightened Black Ale,60 Minute IPA,90 Minute IPA,Aecht Schlenkerla Rauchbier Märzen,AleSmith IPA,AleSmith Speedway Stout,Allagash White,Alpha King Pale Ale
beer_name,,,,,,,,,,
#9,1.000000,0.275405,0.274103,0.388364,0.365175,0.253841,0.228479,0.227612,0.340681,0.293315
120 Minute IPA,0.275405,1.000000,0.251519,0.378258,0.410366,0.262425,0.315971,0.337541,0.282273,0.336796
1554 Enlightened Black Ale,0.274103,0.251519,1.000000,0.319887,0.314028,0.252486,0.266866,0.261761,0.260275,0.307296
60 Minute IPA,0.388364,0.378258,0.319887,1.000000,0.533042,0.316928,0.312343,0.307627,0.360975,0.385249
90 Minute IPA,0.365175,0.410366,0.314028,0.533042,1.000000,0.312861,0.344218,0.358754,0.356804,0.418582
Aecht Schlenkerla Rauchbier Märzen,0.253841,0.262425,0.252486,0.316928,0.312861,1.000000,0.244490,0.246063,0.297672,0.263248
AleSmith IPA,0.228479,0.315971,0.266866,0.312343,0.344218,0.244490,1.000000,0.521889,0.277409,0.400741
AleSmith Speedway Stout,0.227612,0.337541,0.261761,0.307627,0.358754,0.246063,0.521889,1.000000,0.273930,0.420247
Allagash White,0.340681,0.282273,0.260275,0.360975,0.356804,0.297672,0.277409,0.273930,1.000000,0.295666


Select some beers and store them in `beers_i_like` then look their distances to other beers with `head()`

beer_name,Sierra Nevada Pale Ale,120 Minute IPA,Allagash White
beer_name,,,
#9,0.373968,0.275405,0.340681
120 Minute IPA,0.301693,1.000000,0.282273
1554 Enlightened Black Ale,0.330033,0.251519,0.260275
60 Minute IPA,0.459641,0.378258,0.360975
90 Minute IPA,0.441189,0.410366,0.356804


Sum the distances of my favorite beers by row, to have one distance from each beer in the sample.  For instance if there are 3 beers in your `beers_i_like` then you will be summing 3 numbers for each row.  Store the results in `beers_summed`.  There are 2 ways you can do this:  
1. Calling `apply()` with a lambda function that contains `np.sum()` with `axis=1`
2. Calling `np.sum()` with `axis=1` on the entire dataframe (sliced by columns you like)

Optional: which one is faster? use ```%timeit``` to check

10 loops, best of 3: 46.7 ms per loop


100 loops, best of 3: 1.89 ms per loop


Sort summed beers from best to worse using `order()`

beer_name
Sierra Nevada Pale Ale                        1.654205
Allagash White                                1.634784
120 Minute IPA                                1.583966
HopDevil Ale                                  1.224217
Sierra Nevada Celebration Ale                 1.215156
90 Minute IPA                                 1.208359
60 Minute IPA                                 1.198874
Stone Ruination IPA                           1.194210
Stone IPA (India Pale Ale)                    1.193193
Storm King Stout                              1.192405
Arrogant Bastard Ale                          1.189981
Sierra Nevada Bigfoot Barleywine Style Ale    1.178245
Prima Pils                                    1.178093
Brooklyn Black Chocolate Stout                1.156365
Ayinger Celebrator Doppelbock                 1.148356
Hennepin (Farmhouse Saison)                   1.147501
Samuel Adams Boston Lager                     1.146304
Hop Rod Rye                                   1.140271


Filter out the beers used as input using `isin()` and store this in `ranked_beers`, then transform this to a list using `tolist()`.  Print out the first 5 elements.

['HopDevil Ale',
 'Sierra Nevada Celebration Ale',
 '90 Minute IPA',
 '60 Minute IPA',
 'Stone Ruination IPA']

Define a function that does what we just did for an arbitrary input list of beers. it should also receive the maximum number of beers requested n as optional parameter.

In [34]:
def get_similar(beers, n=None):
    """
    calculates which beers are most similar to the beers provided. Does not return
    the beers that were provided
    
    Parameters
    ----------
    beers: list
        some beers!
    
    Returns
    -------
    ranked_beers: list
        rank ordered beers
    """
    

Test your function. Find the 10 beers most similar to "120 Minute IPA"

World Wide Stout
90 Minute IPA
Double Bastard Ale
Stone Ruination IPA
Stone Imperial Russian Stout
Storm King Stout
60 Minute IPA
Oaked Arrogant Bastard Ale
Sierra Nevada Bigfoot Barleywine Style Ale
Brooklyn Black Chocolate Stout


Cool, let's try again with the 10 beers most similar to ["Coors Light", "Bud Light", "Amstel Light"]

1) Miller Lite
2) Budweiser
3) Corona Extra
4) Samuel Adams Boston Lager
5) Heineken Lager Beer
6) Blue Moon Belgian White
7) Guinness Draught
8) Miller High Life
9) Samuel Adams Summer Ale
10) Sierra Nevada Pale Ale


## Movie Recommendations with Recsys
[python-recsys](https://github.com/ocelma/python-recsys) is a nice python library for implementing recommender systems.  We'll use it here to try and make movie recommendations from the [movielens dataset](http://grouplens.org/datasets/movielens/).  

### Install Recsys
First run something like the below code to install everything that you need for recsys.

In [29]:
"""
##install python-recsys

### first install dependencies

pip install csc-pysparse networkx divisi2

### then install recsys
git clone https://github.com/python-recsys/python-recsys.git
cd python-recsys/

python setup.py install

### then Restart Kernel
"""

'\n##install python-recsys\n\n### first install dependencies\n\npip install csc-pysparse networkx divisi2\n\n### then install recsys\ngit clone https://github.com/python-recsys/python-recsys.git\ncd python-recsys/\n\npython setup.py install\n\n### then Restart Kernel\n'

Import `recsys.algorithm`, set `recsys.algorithm.VERBOSE = True` and import `recsys.algorithm.factorize.SVD` class

### Get the Data
Download the movielens dataset from our Google Drive folder [here](https://drive.google.com/folderview?id=0B8EEvWzj1ysxa0JkOERmVWd0a3c&usp=sharing) or from the repo.

Let's look at the files, you can do this however you like.

In [38]:
! ls ../Data/movielens

README      movies.dat  ratings.dat users.dat


Read in the movies.dat data into a variable `movies` by using `pd.read_table` with `sep='::'`.  Make sure to set the `names` to ITEMID, Title, and Genres to set the columns and the `index_col` to ITEMID.

### Explore the Data
Take a look at the movies data with `head()`.

,Title,Genres
ITEMID,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


Load the ratings.dat data into a `ratings` variable with the same separator, and the column names UserID, MovieID, Rating, Timestamp.

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Initialize an `SVD` instance called `svd`

Populate it with the data from the ratings dataset, using the built in `load_data()` method.  You should use `format={'col':0, 'row':1, 'value':2, 'ids': int}` and don't forget the `sep` parameter.

Loading ../Data/movielens/ratings.dat
..........|


Compute SVD with a call to `svd.compute()`.  
- Use `k=100`
- Use `min_values=10`
- Use `pre_normalize=None`
- Use `mean_center=True`
- Use `post_normalize=True`

$M=U \Sigma V^T$:

Creating matrix (1000209 tuples)
Matrix density is: 4.4684%
Updating matrix: squish to at least 10 values
Computing svd k=100, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True
[WARNING] mean_center is True. svd.similar(...) might return nan's. If so, then do svd.compute(..., mean_center=False)


you can also save the output SVD model (in a zip file)

In [52]:
# svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True, savefile='/tmp/movielens')

Reload a saved model:

In [53]:
# svd2 = SVD(filename='/tmp/movielens')

### Computing Similarities and Making Recommendations
Let's compute similarity between two movies, first we need to use the movies table to get the itemid that will be used with the ratings data that generated our svd.

Determine the movie ids of "Toy Story (1995)" and "Bug's Life, A (1998)".

,Title,Genres
ITEMID,,
1,Toy Story (1995),Animation|Children's|Comedy


,Title,Genres
ITEMID,,
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy


Print the similarity of these 2 movies by calling `svd.similarity()` with those 2 IDs.

0.677069366773


Use `svd.similar()` to get movies similar to Toy Story.

[(1, 0.99999999999999978),
 (3114, 0.87060391051017305),
 (2355, 0.67706936677314977),
 (588, 0.58073514967544992),
 (595, 0.46031829709744226),
 (1907, 0.44589398718134982),
 (364, 0.42908159895577563),
 (2081, 0.42566581277822413),
 (3396, 0.42474056361934953),
 (2761, 0.40439361857576017)]

Try using `svd.predict()` to predict ratings for a given user and movie, $\hat{r}_{ui}$

5.0

Look it up in the matrix...

In [52]:
svd.get_matrix().value(ITEMID, USERID)

5.0

Try using `svd.recommend()` to Recommend non rated movies to a user (`is_row=False`)

[(2028, 5.4018452642332546),
 (527, 5.3498144196809516),
 (2905, 5.2133848204673132),
 (318, 5.2052108435955446),
 (1193, 5.1942189963876562),
 (3114, 5.1753939214583697),
 (1, 5.1714259073839521),
 (2019, 5.1037438278754719),
 (1178, 5.0962756861446641),
 (1207, 5.090305272922329)]

Which users should see Toy Story? (e.g. which users -that have not rated Toy Story- would give it a high rating?)

[(869, 6.8215500393190904),
 (4086, 6.2667649038936908),
 (549, 6.2394061595542869),
 (1343, 6.2163075783431427),
 (1586, 6.039893928886932),
 (840, 5.9616632765170472),
 (1676, 5.896233772781037),
 (4595, 5.88945710113423),
 (2691, 5.8735094161364714),
 (2665, 5.8498694241604259)]

Find out more here: [https://github.com/ocelma/python-recsys](https://github.com/ocelma/python-recsys)